# Visualizing Eye Tracking
As a measure of behavioral information, the Allen Institute tracks mouse eye motion during experiments, as well as other things like eye area and blink events. In this notebook, this information is extracted from an NWB file and visualized with matplotlib.

### Environment Setup

In [ ]:
from pynwb import NWBHDF5IO
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from dandi import dandiapi

%matplotlib inline

### Downloading NWB Files
To choose your own files to download, set `dandiset_id`, `dandi_stim_filepath`, `dandi_lfp_filepath` accordingly.

In [ ]:
dandiset_id = "000021"
dandi_filepath = "sub-1169714184/sub-1169714184_ses-1180116198.nwb"
download_loc = "./"
authenticate = False
dandi_api_key = ""

In [ ]:
filename = dandi_filepath.split("/")[-1]
if authenticate:
    my_dandiset = dandiapi.DandiAPIClient(token=dandi_api_key).get_dandiset(dandiset_id)
else:
    my_dandiset = dandiapi.DandiAPIClient().get_dandiset(dandiset_id)
print(f"Got dandiset {my_dandiset}")

In [ ]:
file = my_dandiset.get_asset_by_path(dandi_filepath)
# this may take awhile, especially if the file to download is large
file.download(f"{download_loc}/{filename}")

print(f"Downloaded file to {download_loc}/{filename}")

### Extracting Eye Tracking Data
Our datasets include eye data with eye tracking, corneal reflection tracking, and pupil tracking. Any of these should be usable for the following analyses. Below, you can set `eye_tracking` to one of those values commented out below. You can also take the probable blink times.

In [ ]:
stim_filepath = f"{download_loc}/{filename}"

In [ ]:
io = NWBHDF5IO(stim_filepath, mode="r", load_namespaces=True)
nwb = io.read() 

In [ ]:
eye_tracking = nwb.acquisition["EyeTracking"].eye_tracking
# eye_tracking = nwb.acquisition["EyeTracking"].corneal_reflection_tracking
# eye_tracking = nwb.acquisition["EyeTracking"].pupil_tracking

timestamps = eye_tracking.timestamps
blink_times = nwb.acquisition["EyeTracking"].likely_blink

print(eye_tracking)

### Selecting a Period
The data can be large or messy. In order to visualize the data more cleanly and efficiently, you can just select a period of time within the data to plot. To do this, specify the `start_time` and `end_time` you'd like in terms of seconds. Below, the first and last timestamps from the data are printed to assist in this choice.

In [ ]:
print(timestamps[0])
print(timestamps[-2])

In [ ]:
start_time = 0
end_time = 100

In [ ]:
# get data indices from times

start_idx, end_idx = None, None
for i, ts in enumerate(timestamps):
    if not start_idx and ts >= start_time:
        start_idx = i
    if start_idx and ts >= end_time:
        end_idx = i
        break

if start_idx == None or end_idx == None:
    raise ValueError("Time bounds not found within eyetracking timestamps")

In [ ]:
# make time axis
time_axis = np.arange(start_idx, end_idx)

### Visualizing the Data
Below, several types of measurements are visualized from the eye tracking data that was selected.

#### Blink Times

In [ ]:
fig, ax = plt.subplots()
ax.plot(blink_times.data[start_idx:end_idx], linewidth=0.2)

#### Area
Below, eye height and width are plotted together, and area, the product of height and width, is also plotted.

In [ ]:
fig, ax1 = plt.subplots()
ax1.set_xlabel('time')
ax1.plot(time_axis, eye_tracking.width[start_idx:end_idx], color='b')
ax1.set_ylabel('width', color='b')
ax2 = ax1.twinx()
ax2.plot(time_axis, eye_tracking.height[start_idx:end_idx], color='r')
ax2.set_ylabel('height', color='r')

In [ ]:
fig, ax1 = plt.subplots()
ax1.set_xlabel('time')
ax1.set_ylabel('area')
ax1.plot(time_axis, eye_tracking.area[start_idx:end_idx])

#### Angle

In [ ]:
fig, ax = plt.subplots()
angle = np.array(eye_tracking.angle)
ax.set_xlabel('time')
ax.set_ylabel('angle (degrees)')
ax.plot(time_axis, angle[start_idx:end_idx])

#### Eye Trace
With marker color representing time, the x and y coordinates of the eye's view are traced below.

In [ ]:
# extract coords from eye tracking array
xs = np.array([point[0] for point in eye_tracking.data])
ys = np.array([point[1] for point in eye_tracking.data])

In [ ]:
fig, ax = plt.subplots()
colors = plt.cm.viridis(np.linspace(0, 1, end_idx-start_idx))
ax.plot(xs[start_idx:end_idx], ys[start_idx:end_idx], zorder=0, linewidth=0.25)
ax.scatter(xs[start_idx:end_idx], ys[start_idx:end_idx], s=5, c=colors, zorder=1)

# set these to restrict the plot limits
# ax.set_xlim(320,350)
# ax.set_ylim(270,320)
plt.show()